In [1]:
%matplotlib inline

import glob
from string import letters
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from scipy import signal
from sklearn.decomposition import PCA

//anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


We want to create a locations dataframe with heterochromatic state.

### Updating chromatin state model to right version

First we load chromatin state file (this is for an old drosophila genome version).  We're going to want to update it to the right version.

In [2]:
chromatin_df=pd.read_csv("GSE22069_Drosophila_chromatin_domains.txt", sep='\t')
chromatin_df.head()

,seqname,start,end,chromatin
0,chr2L,5301,58754,BLACK
1,chr2L,58763,65319,BLUE
2,chr2L,65316,76559,RED
3,chr2L,76693,84511,BLUE
4,chr2L,84650,89599,RED


Then we create a column that formats the information such that it can be inputted in http://flybase.org/static_pages/downloads/COORD.html .  We write this column to 20160625_Chromatin_state_v4_to_v6 

In [3]:
#chromatin_df["ForUpdate"]=chromatin_df.seqname[3:] + ':' + str(chromatin_df.start) + '..' + str(chromatin_df.end)
#chromatin_df["ForUpdate"]=chromatin_df.seqname.astype(str).apply(lambda x: x[3:]) + ':' +chromatin_df.start.astype(str)+ '..' + chromatin_df.end.astype(str)
#chromatin_df.head()
#chromatin_df.ForUpdate.to_csv("20160625_Chromatin_state_v4_to_v6", index=False)

We update this file to the right version (version 6) using  http://flybase.org/static_pages/downloads/COORD.html.  We write the new version 6 results to the following file:
"FlyBase_Converted_Coordinates.txt".  

In the next block, we format the new updated chromatin_df dataframe.

In [4]:
Updated_chromatin_df = pd.read_csv("FlyBase_Converted_Coordinates.txt", sep="\t")
Updated_chromatin_df["seqname"] = Updated_chromatin_df.Converted.astype(str).apply(lambda x: x.split(':')[0])
Updated_chromatin_df["start"] = Updated_chromatin_df.Converted.astype(str).apply(lambda x: x.split(':')[1].split('..')[0])
Updated_chromatin_df["end"] = Updated_chromatin_df.Converted.astype(str).apply(lambda x: x.split(':')[1].split('..')[1])
del Updated_chromatin_df["Converted"]

Updated_chromatin_df.index = range(Updated_chromatin_df.shape[0])
Updated_chromatin_df["chromatin"] = chromatin_df.chromatin
Updated_chromatin_df.head()

,seqname,start,end,chromatin
0,2L,5301,58754,BLACK
1,2L,58763,65319,BLUE
2,2L,65316,76559,RED
3,2L,76693,84511,BLUE
4,2L,84650,89599,RED


In [5]:
del chromatin_df 

### Adding state to locations df

First we load the locations_df

In [6]:
locations_df=pd.read_csv("Locations")
locations_df.columns = ["a", "chr","start","end"]
del locations_df["a"]
locations_df.head()

,chr,start,end
0,211000022278049,390,394
1,211000022278049,537,541
2,211000022278049,573,577
3,211000022278049,837,841
4,211000022278049,1613,1617


In [ ]:
locations_df["chromatin_state"]=[np.nan for ele in locations_df['chr']]
for i,row1 in enumerate(locations_df.iterrows()):
    if not i%1000:
        print i/1000,
    key = str(row1[1]['chr'])
    #print row1
    subset_chrom_df=Updated_chromatin_df[Updated_chromatin_df['seqname'] == key]
    len_subset_chrom_df = subset_chrom_df.shape[0]
    #print len_subset_chrom_df
    for j in range(len_subset_chrom_df):
        try:
            if float(row1[1]['end']) <= float(subset_chrom_df.loc[subset_chrom_df.index[j],'end']) and float(row1[1]['start']) >= float(subset_chrom_df.loc[subset_chrom_df.index[j],'start']):
                    #print subset_chrom_df.loc[subset_chrom_df.index[j],"chromatin"]
                    locations_df.ix[i,'chromatin_state'] = subset_chrom_df.ix[subset_chrom_df.index[j],'chromatin']
                    break
        except:
            pass
            #print j,
            #print subset_chrom_df.loc[subset_chrom_df.index[j]]

locations_df.head(n=20)

0 1 2 3 4 5 6 7 8 

Do some exploratory analysis

In [17]:
len(locations_df)

388464

### In what chromatin state are differentially expressed genes?

First we load a list of the regions of differentially expressed genes (for carcass data).  [You need to specify where you got this list from]

In [10]:
diff_expression_locations=[]
fp=open("FlyBase_FastA_differentially_expressed_carcass.txt", "r")
for line in fp:
    if line[0]=='>':
        diff_expression_locations.append(line.split(";")[1])
fp.close()
print len(diff_expression_locations)

443


In [11]:
diff_expression_locations[:10]

[' loc=X:245978..254650',
 ' loc=X:complement(585081..620195)',
 ' loc=X:complement(946662..947500)',
 ' loc=X:complement(2278031..2293411)',
 ' loc=X:complement(2622069..2624575)',
 ' loc=X:2948034..2949120',
 ' loc=X:complement(2885186..2885914)',
 ' loc=X:4939128..4955916',
 ' loc=X:5421533..5427219',
 ' loc=X:complement(6349016..6350401)']

We remove the word complement, and create a dataframe of these values.

In [49]:
diff_expression_locations =[ele.replace('complement(','') for ele in diff_expression_locations]
diff_expression_locations = [ele.strip(')') for ele in diff_expression_locations]

diff_expression_df=pd.DataFrame()
diff_expression_df["chr"]=[ele.split(":")[0].split("=")[1] for ele in diff_expression_locations]
diff_expression_df["start"]=[ele.split(":")[1].split("..")[0] for ele in diff_expression_locations]
diff_expression_df["end"]=[ele.split("..")[1] for ele in diff_expression_locations]


diff_expression_df.to_csv("Differentially_expressed_genes_carcass_df")

diff_expression_df.head()

,chr,start,end
0,X,245978,254650
1,X,585081,620195
2,X,946662,947500
3,X,2278031,2293411
4,X,2622069,2624575


Now, we use the same process as before to find the chromatin state of each of these differentially bound regions

In [23]:
diff_expression_df["chromatin_state"]=[np.nan for ele in diff_expression_df['chr']]
for i,row1 in enumerate(diff_expression_df.iterrows()):
    print i,
    key = str(row1[1]['chr'])     
    subset_chrom_df=Updated_chromatin_df[Updated_chromatin_df.seqname == key]
    len_subset_chrom_df = subset_chrom_df.shape[0]
    for j in range(len_subset_chrom_df):
        try:
            if float(row1[1]['end']) <= float(subset_chrom_df.loc[subset_chrom_df.index[j],'end']) and float(row1[1]['start']) >= float(subset_chrom_df.loc[subset_chrom_df.index[j],'start']) :
                    diff_expression_df.ix[i,'chromatin_state'] = subset_chrom_df.ix[subset_chrom_df.index[j],'chromatin']
                    break
        except:
            pass


diff_expression_df.head(n=20)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

,chr,start,end,chromatin_state
0,X,245978,254650,BLACK
1,X,585081,620195,NaN
2,X,946662,947500,NaN
3,X,2278031,2293411,NaN
4,X,2622069,2624575,YELLOW
5,X,2948034,2949120,BLACK
6,X,2885186,2885914,BLACK
7,X,4939128,4955916,NaN
8,X,5421533,5427219,BLACK
9,X,6349016,6350401,YELLOW


In [24]:
diff_expression_df.groupby("chromatin_state").count()

,chr,start,end
chromatin_state,,,
BLACK,128,128,128
BLUE,43,43,43
GREEN,9,9,9
RED,10,10,10
YELLOW,84,84,84


In [47]:
sum([1 for ele in diff_expression_df["chromatin_state"] if str(ele)=="nan"])

169

Interesting as well to think of the number of NaN, because this gives you the guys in between chromatin states, which might be reminiscent of PEV.  You need to do some more analysis.  

a large number of differentially expressed genes are between chromatin states